<a href="https://colab.research.google.com/github/Lucas-Ellenberger/CSE144/blob/main/stage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os

In [2]:
# make sure you're logged in with `huggingface-cli login`
from huggingface_hub import login

login()

In [3]:
!pip install torch
!pip install diffusers
!pip install accelerate
!pip install datasets
!pip install datasets[vision]

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [4]:
import torch
import pandas
from torch import autocast
import accelerate
from diffusers import StableDiffusionPipeline

In [5]:
from diffusers import DDPMPipeline
from accelerate import Accelerator
from datasets import load_dataset

In [6]:
# !pip install datasets
# !pip install datasets[vision]
# from datasets import load_dataset
dataset = load_dataset("AI4Math/MathVista")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating testmini split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5141 [00:00<?, ? examples/s]

In [ ]:
# Use a pipeline as a high-level helper
# from transformers import pipeline

# pipe = pipeline("image-to-text", model="Salesforce/blip-image-captioning-large")

In [ ]:
# import requests
# from PIL import Image
# from transformers import BlipProcessor, BlipForConditionalGeneration

# processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg'
# raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

# # conditional image captioning
# text = "a photography of"
# inputs = processor(raw_image, text, return_tensors="pt")

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))

# # unconditional image captioning
# inputs = processor(raw_image, return_tensors="pt")

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))


In [7]:
# print the first example on the testmini set
print(dataset["testmini"][0])
print(dataset["testmini"][0]['pid']) # print the problem id
print(dataset["testmini"][0]['question']) # print the question text
print(dataset["testmini"][0]['query']) # print the query text
print(dataset["testmini"][0]['image']) # print the image path
print(dataset["testmini"][0]['answer']) # print the answer
dataset["testmini"][0]['decoded_image'] # display the image

# print the first example on the test set
print(dataset["test"][0])

{'pid': '1', 'question': "When a spring does work on an object, we cannot find the work by simply multiplying the spring force by the object's displacement. The reason is that there is no one value for the force-it changes. However, we can split the displacement up into an infinite number of tiny parts and then approximate the force in each as being constant. Integration sums the work done in all those parts. Here we use the generic result of the integration.\r\n\r\nIn Figure, a cumin canister of mass $m=0.40 \\mathrm{~kg}$ slides across a horizontal frictionless counter with speed $v=0.50 \\mathrm{~m} / \\mathrm{s}$. It then runs into and compresses a spring of spring constant $k=750 \\mathrm{~N} / \\mathrm{m}$. When the canister is momentarily stopped by the spring, by what distance $d$ is the spring compressed?", 'image': 'images/1.jpg', 'decoded_image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1514x720 at 0x796C77C245E0>, 'choices': None, 'unit': None, 'precision': 1.0

In [ ]:
# Uncomment if you want to see a sample image from the dataset
# categories = set()
# count = 0
# descriptions = []
# # for i in range(len(dataset['testmini'])):
# for i in range(len(dataset['testmini'])):
#     inputs = processor(dataset["testmini"][i]['decoded_image'], return_tensors="pt")
#     out = model.generate(**inputs)
#     descriptions.append(processor.decode(out[0], skip_special_tokens=True))


# # print(descriptions)
# addition = {
#     'description': descriptions
# }
# print("count: ", count)
# print("len(testmini): ", len(dataset['testmini']))
# print(count / len(dataset['testmini']))
# print(categories)
# print(len(dataset['test']))
# i = 8
# print(dataset['testmini'][i]['metadata']['category'])
# print(dataset["testmini"][i]['query']) # print the question text
# unconditional image captioning
# inputs = processor(dataset["testmini"][i]['decoded_image'], return_tensors="pt")

# out = model.generate(**inputs)
# print(processor.decode(out[0], skip_special_tokens=True))
# dataset['testmini'][i]['decoded_image']

/home/lucas_ellenberger/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['diagram of a spring force does negative work, decreasing speed and kinetic energy', 'there is a bowl of flour and eggs on a cutting board', 'a drawing of a triangle with three intersecting sides', 'there are three different colored objects on a white surface', 'a picture of a diagram of a right triangle with a right angle', 'a black and white photo of a number of letters on a line', 'a close up of a clock with a purple background and a white face', 'there are many different types of toys that are on the table', 'a picture of a box with a letter a and a box with a letter b', 'a close up of a cell phone with a number of letters', 'there are many different colored objects arranged in a circle', 'there is a bike and a kite flying in the air', 'arafed graph of different types of objects in different groups', 'there are many toy cars and a car on the table', 'a diagram of a water cycle with different types of water', 'a diagram of sales statistics for items in different stores', 'a picture

In [ ]:
# df = pandas.DataFrame.from_dict(addition)4
# import os
# os.makedirs('prompts/testmini', exist_ok=True)
# df.to_csv('prompts/testmini/prompts.csv')

In [ ]:
# train_prompts = []

# for i in range(len(dataset['test'])):
#     inputs = processor(dataset["test"][i]['decoded_image'], return_tensors="pt")
#     out = model.generate(**inputs)
#     train_prompts.append(processor.decode(out[0], skip_special_tokens=True))
#     if i % 500 == 0:
#         print("Completed iter: ", i)

/home/lucas_ellenberger/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Completed iter:  0
Completed iter:  500
Completed iter:  1000
Completed iter:  1500
Completed iter:  2000
Completed iter:  2500
Completed iter:  3000
Completed iter:  3500
Completed iter:  4000
Completed iter:  4500
Completed iter:  5000


In [ ]:
# train_prompts_dict = {
#     'train_prompts' : train_prompts
# }

# prompts = pandas.DataFrame.from_dict(train_prompts_dict)
# os.makedirs('prompts/test', exist_ok=True)
# prompts.to_csv('prompts/test/prompts.csv')

In [9]:
import pandas

testmini_prompts_df = pandas.read_csv("prompts.csv")
test_prompts_df = pandas.read_csv("test-prompts.csv")

In [13]:
# print(len(test_prompts_df))
testmini_prompts_df

,Unnamed: 0,description
0,0,"diagram of a spring force does negative work, ..."
1,1,there is a bowl of flour and eggs on a cutting...
2,2,a drawing of a triangle with three intersectin...
3,3,there are three different colored objects on a...
4,4,a picture of a diagram of a right triangle wit...
...,...,...
995,995,there are three different colored cubes and a ...
996,996,a diagram of a curve with a line of length and...
997,997,a diagram of a tree with three branches and th...
998,998,a diagram of a curve with a y and a y on it


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
pipe = StableDiffusionPipeline.from_pretrained(
	"CompVis/stable-diffusion-v1-4",
	use_auth_token=True
).to(device)
i = 0
prompt = testmini_prompts_df['description'][i]
output = "testmini_%d.png" % i
# with autocast(device):
image = pipe(prompt).images[0]

image.save(output)

Keyword arguments {'use_auth_token': True} are not expected by StableDiffusionPipeline and will be ignored.


cuda


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [18]:
for i in range(1, 20):
    prompt = testmini_prompts_df['description'][i]
    output = "testmini_%d.png" % i
    image = pipe(prompt).images[0]
    image.save(output)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [19]:
for i in range(0, 20):
    filename = "testmini_prompt%d.txt" % i
    prompt = testmini_prompts_df['description'][i]
    f = open(filename, "a")
    f.write(prompt)
    f.close()